##### Copyright 2024 Google LLC.

@title Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Day 1 - Prompting

Welcome to the Kaggle 5-day Generative AI course!

This notebook will show you how to get started with the Gemini API and walk you through some of the example prompts and techniques that you can also read about in the Prompting whitepaper. You don't need to read the whitepaper to use this notebook, but the papers will give you some theoretical context and background to complement this interactive notebook.


## Before you begin

In this notebook, you'll start exploring prompts and prompt parameters using the Python SDK and AI Studio. For some inspiration, you might enjoy exploring some apps that have been built using the Gemini family of models. Here are a few that we like, and we think you will too.

* [TextFX](https://textfx.withgoogle.com/) is a suite of AI-powered tools for rappers, made in collaboration with Lupe Fiasco,
* [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) shows how you can talk directly to a database using the Gemini API,
* [NotebookLM](https://notebooklm.google/) uses Gemini models to build your own personal AI research assistant.


## For help

**Common issues are covered in the [FAQ and troubleshooting guide](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs).**

### A note on the Gemini API and Vertex AI

In the whitepapers, most of the example code uses the Enterprise [Vertex AI platform](https://cloud.google.com/vertex-ai). In contrast, this notebook, along with the others in this series, will use the [Gemini Developer API](https://ai.google.dev/gemini-api/) and [AI Studio](https://aistudio.google.com/).

Both APIs provide access to the Gemini family of models, and the code to interact with the models is very similar. Vertex provides a world-class platform for enterprises, governments and advanced users that need powerful features like data governance, ML ops and deep Google Cloud integration.

AI Studio is free to use and only requires a compatible Google account to log in and get started. It is deeply integrated with the Gemini API, which comes with a generous [free tier](https://ai.google.dev/pricing) that you can use to run the code in these exercises.

If you are already set up with Google Cloud, you can check out the [Enterprise Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference) through Vertex AI, and run the samples directly from the supplied whitepapers.

### Install the SDK

In [1]:
%pip install -U -q "google-generativeai>=0.8.3"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display
from env import *

%load_ext autoreload
%autoreload 2

### Set up API key


In [3]:
GOOGLE_API_KEY = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

### List of available models

In [ ]:
# Retrieving the names of available models
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/embedding-001
models/text-embedding-004
models/aqa


In [ ]:
# Retrieving a detailed description of the models for comparison

import pandas as pd
import re

model_df = pd.DataFrame(columns = ['name',
                                    'base_model_id',
                                    'version',
                                    'display_name',
                                    'description',
                                    'input_token_limit',
                                    'output_token_limit',
                                    'supported_generation_methods',
                                    'temperature',
                                    'max_temperature',
                                    'top_p',
                                    'top_k'])

pattern = r"(\w+)\s*=\s*(.*?)(?=,\s*\w+=|\)$)"

for model in genai.list_models():
    model_text = str(model)
    model_dict = {match[0]: match[1].strip() for match in re.findall(pattern, model_text)}
    model_dict_listified = {key:[value] for key, value in model_dict.items()}
    model_df = pd.concat([model_df,pd.DataFrame(model_dict_listified)], ignore_index= True)

model_df.drop(columns = ['base_model_id'], inplace = True) # dropping as the values are null
model_df

,name,version,display_name,description,input_token_limit,output_token_limit,supported_generation_methods,temperature,max_temperature,top_p,top_k
0,'models/chat-bison-001','001','PaLM 2 Chat (Legacy)','A legacy text-only model optimized for chat c...,4096,1024,"['generateMessage', 'countMessageTokens']",0.25,None,0.95,40
1,'models/text-bison-001','001','PaLM 2 (Legacy)','A legacy model that understands text and gene...,8196,1024,"['generateText', 'countTextTokens', 'createTun...",0.7,None,0.95,40
2,'models/embedding-gecko-001','001','Embedding Gecko','Obtain a distributed representation of a text.',1024,1,"['embedText', 'countTextTokens']",None,None,None,None
3,'models/gemini-1.0-pro-latest','001','Gemini 1.0 Pro Latest',NaN,30720,2048,"['generateContent', 'countTokens']",0.9,None,1.0,None
4,'models/gemini-1.0-pro','001','Gemini 1.0 Pro','The best model for scaling across a wide rang...,30720,2048,"['generateContent', 'countTokens']",0.9,None,1.0,None
5,'models/gemini-pro','001','Gemini 1.0 Pro','The best model for scaling across a wide rang...,30720,2048,"['generateContent', 'countTokens']",0.9,None,1.0,None
6,'models/gemini-1.0-pro-001','001','Gemini 1.0 Pro 001 (Tuning)',NaN,30720,2048,"['generateContent', 'countTokens', 'createTune...",0.9,None,1.0,None
7,'models/gemini-1.0-pro-vision-latest','001','Gemini 1.0 Pro Vision',NaN,12288,4096,"['generateContent', 'countTokens']",0.4,None,1.0,32
8,'models/gemini-pro-vision','001','Gemini 1.0 Pro Vision',NaN,12288,4096,"['generateContent', 'countTokens']",0.4,None,1.0,32
9,'models/gemini-1.5-pro-latest','001','Gemini 1.5 Pro Latest',NaN,2000000,8192,"['generateContent', 'countTokens']",1.0,2.0,0.95,40


A more detailed characteristics of the models can be found <a href='https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-flash'>here<a>

### Output text generation

By using the `gemini-1.5-flash` model.

In [ ]:
flash_model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
# Text generation, first with text stream showing partial results, second with markdown
response = flash_model.generate_content("Explain how nuclear reactor works, without being too much technical",
                                        stream=True)
for chunk in response:
    print(chunk.text, end=" ")
    print("...", end = '')
Markdown(response.text)

Imagine ... a really, really hot fire that's fueled by splitting atoms instead of wood ... or gas. That's basically a nuclear reactor.

Here's the ... simplified process:

1. **Fuel:**  The reactor uses special materials like uranium, which contains atoms with unstable nuclei.  These nuclei are like tiny, ... tightly wound springs that are ready to unleash a lot of energy.

2. **Splitting Atoms (Fission):**  We carefully control the splitting of ... these unstable atoms.  When an atom splits (fission), it releases a huge amount of heat – much more than burning any normal fuel.  It also releases tiny particles called neutrons.

3. **Chain Reaction:**  These neutrons ... then go on to hit other unstable atoms, causing *them* to split, releasing even more heat and more neutrons. This creates a self-sustaining chain reaction – a controlled explosion of sorts.

4. **Heat Transfer:** The intense ... heat generated is used to boil water.

5. **Electricity Generation:** The steam from the boil

Imagine a really, really hot fire that's fueled by splitting atoms instead of wood or gas. That's basically a nuclear reactor.

Here's the simplified process:

1. **Fuel:**  The reactor uses special materials like uranium, which contains atoms with unstable nuclei.  These nuclei are like tiny, tightly wound springs that are ready to unleash a lot of energy.

2. **Splitting Atoms (Fission):**  We carefully control the splitting of these unstable atoms.  When an atom splits (fission), it releases a huge amount of heat – much more than burning any normal fuel.  It also releases tiny particles called neutrons.

3. **Chain Reaction:**  These neutrons then go on to hit other unstable atoms, causing *them* to split, releasing even more heat and more neutrons. This creates a self-sustaining chain reaction – a controlled explosion of sorts.

4. **Heat Transfer:** The intense heat generated is used to boil water.

5. **Electricity Generation:** The steam from the boiling water spins turbines, which in turn generate electricity.

6. **Control Rods:**  To control the speed of the chain reaction (and prevent it from getting out of control), special rods made of materials that absorb neutrons are inserted into the reactor core.  By adjusting the position of these control rods, the reactor's power output can be precisely regulated.

In short, a nuclear reactor uses the energy released from splitting atoms to create heat, which is then used to generate electricity. It's a very efficient process, but requires extremely careful control to ensure safety.


In [64]:
# Chat generation
chat = flash_model.start_chat(history=[
    {'role': 'user' , 'parts' : "Try to guess a fruit, in two questions"}
])
response = chat.send_message("I'll start with a hint : it's a tropical fruit")
print(response.text)

Okay, my first question is:  Is it typically eaten peeled?



In [65]:
response = chat.send_message('Yes')
print(response.text)

My second question: Is it primarily yellow or orange when ripe?



In [66]:
response = chat.send_message('Yes')
print(response.text)

My guess is a mango.



In [ ]:
response = chat.send_message('Yes')

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Excellent!  I'm glad we could play.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.20676344633102417
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 81,
        "candidates_token_count": 12,
        "total_token_count": 93
      }
    }),
)

In [73]:
#Show chat history
chat.history

[parts {
   text: "Try to guess a fruit, in two questions"
 }
 role: "user",
 parts {
   text: "I\'ll start with a hint : it\'s a tropical fruit"
 }
 role: "user",
 parts {
   text: "Okay, my first question is:  Is it typically eaten peeled?\n"
 }
 role: "model",
 parts {
   text: "Yes"
 }
 role: "user",
 parts {
   text: "My second question: Is it primarily yellow or orange when ripe?\n"
 }
 role: "model",
 parts {
   text: "Yes"
 }
 role: "user",
 parts {
   text: "My guess is a mango.\n"
 }
 role: "model",
 parts {
   text: "Yes"
 }
 role: "user",
 parts {
   text: "Great!  I got it.\n"
 }
 role: "model",
 parts {
   text: "Yes"
 }
 role: "user",
 parts {
   text: "Excellent!  I\'m glad we could play.\n"
 }
 role: "model"]

## Explore generation parameters



### Output length

Using `max_output_tokens` parameter, and prompt engineering. Note that output token does not equal word counts.

In [74]:
short_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(max_output_tokens=200))

response = short_model.generate_content('Write a 198 word essay on the importance of chewing gum in modern society.')
print(response.text)

Chewing gum, a seemingly trivial item, holds surprising significance in modern society. Beyond its simple pleasure, it plays a multifaceted role, impacting oral health, cognitive function, and even social interactions.

Firstly, chewing gum stimulates saliva production, a natural defense against tooth decay. This increased saliva flow neutralizes acids, washes away food particles, and strengthens tooth enamel.  Many gums now incorporate xylitol, a sugar alcohol proven to inhibit bacterial growth, further bolstering oral hygiene.

Secondly, the act of chewing can improve concentration and alertness. Studies suggest that the rhythmic jaw movements stimulate blood flow to the brain, enhancing focus and reducing stress. This makes chewing gum a useful tool for students during exams or for individuals needing a mental boost.

Finally, chewing gum has social implications.  It serves as a subtle, socially acceptable fidgeting mechanism, alleviating anxiety in stressful situations. The act of 

### Temperature

`temperature` parameter is used to define the degree of randomness in selecting tokens. 

In case of `gemini-1.5-flash` model, default value is set at 1.0, with 2.0	as the possible maximum value. 

Higher the temperature, higher number of candidate tokens to select from, resulting in more open ended and creative results. On the other hand, lower temperature restricts the number of possible tokens. Temperature 0, also referred to as 'greedy coding', selects the token with the highest probability.  

To note that temperature doesn't provide any guarantees of randomness, but it can be used to somewhat "nudge" the output.

In [ ]:
from google.api_core import retry

max_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=2.0,
                                             max_output_tokens=100))

avg_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=1.0,
                                             max_output_tokens=100))

min_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0,
                                             max_output_tokens=100))

# When running lots of queries, it's a good practice to use a retry policy so your code
# automatically retries when hitting Resource Exhausted (quota limit) errors.
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}

instruction = """Continue the story and add two sentences. New line for each sentence :
                It was a bright cold day in April, and the clocks were striking thirteen."""

print(f"Opening line from the book 1984 by George Orwell : \
      \n It was a bright cold day in April, and the clocks were striking thirteen.\n {'-'*25}")

for temp in range(3):
    model = genai.GenerativeModel('gemini-1.5-flash',
                                  generation_config=genai.GenerationConfig(temperature=temp,
                                                                        max_output_tokens=100))
    print(f'Temperature value : {temp}')
    response = model.generate_content(instruction, request_options=retry_policy)

    if response.parts:
        print(response.text, '-' * 25)

Opening line from the book 1984 by George Orwell :       
 It was a bright cold day in April, and the clocks were striking thirteen.
 -------------------------
Temperature value : 0
Winston Smith, his chin nuzzled into his threadbare collar, hurried through the streets of London.  
A telescreen, its ever-watchful eye gleaming, seemed to follow his every move.
 -------------------------
Temperature value : 1
Winston Smith, his chin nuzzled into his threadbare collar, hurried through the streets of London.  
A telescreen's watchful eye seemed to follow him from every shadowed doorway.
 -------------------------
Temperature value : 2
Winston Smith, his chin nuzzled into his threadbare collar, hurried down the street.
A telescreen, its ever-watchful eye gleaming, seemed to follow his every move.
 -------------------------


Interesting how the sentences begins with the same phrase "Winston Smith, his chin nuzzled into his threadbare collar, hurried", no matter the value of the temperature. Structure of the sentences also resembles the same. 

In [21]:
low_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0))

for _ in range(5):
  response = low_temp_model.generate_content('Pick a random colour... (respond in a single word)',
                                             request_options=retry_policy)
  if response.parts:
    print(response.text, '-' * 25)

Purple 
 -------------------------
Purple 
 -------------------------
Purple 
 -------------------------
Purple 
 -------------------------
Purple 
 -------------------------


### Top-K and top-P

Like temperature, top-K and top-P parameters are also used to control the diversity of the model's output.

Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.

Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.

When both are supplied, the Gemini API will filter top-K tokens first, then top-P and then finally sample from the candidate tokens using the supplied temperature.

Run this example a number of times, change the settings and observe the change in output.

In [25]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        # These are the default values for gemini-1.5-flash-001.
        temperature=0,
        top_k=6,
        top_p=0.95,
    ))

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = model.generate_content(story_prompt, request_options=retry_policy)
print(response.text)

Bartholomew, a ginger tabby with a penchant for mischief and a disdain for routine, was bored. The sunbeams that usually danced on his favorite rug were dull, the birds outside chirping the same monotonous tune. He yearned for something more, something… adventurous.

His opportunity arrived in the form of a delivery truck. The driver, a burly man with a booming laugh, carelessly left the back door ajar. Bartholomew, with the agility of a seasoned acrobat, slipped inside. The truck rumbled to life, and Bartholomew found himself on a journey unlike any he'd ever known.

The world outside his usual window was a kaleidoscope of sights and smells. He saw towering buildings that scraped the sky, bustling markets overflowing with exotic fruits, and a vast, shimmering ocean. He sniffed the scent of fresh bread, salty air, and the sweet perfume of blooming flowers.

His adventure took him to a bustling city park, where he encountered a group of pigeons. They were wary at first, but Bartholomew,

## Prompting

This section contains some prompts from the chapter for you to try out directly in the API. Try changing the text here to see how each prompt performs with different instructions, more examples, or any other changes you can think of.

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [27]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

Sentiment: **POSITIVE**


#### Enum mode

The models are trained to generate text, and can sometimes produce more text than you may wish for. In the preceding example, the model will output the label, sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows you to constrain the output to a fixed set of values.

In [28]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ))

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

positive


### One-shot and few-shot

Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1jjWkjUSoMXmLvMJ7IzADr_GxHPJVV2bg"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [29]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"


response = model.generate_content([few_shot_prompt, customer_order], request_options=retry_policy)
print(response.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
``` 



#### JSON mode

To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [30]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ))

response = model.generate_content("Can I have a large dessert pizza with apple and chocolate")
print(response.text)

{"ingredients": ["apple", "chocolate"], "size": "large", "type": "dessert"}



### Chain of Thought (CoT)

Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.

Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

As models like the Gemini family are trained to be "chatty" and provide reasoning steps, you can ask the model to be more direct in the prompt.

In [31]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
response = model.generate_content(prompt, request_options=retry_policy)

print(response.text)

52 



Now try the same approach, but indicate to the model that it should "think step by step".

In [32]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = model.generate_content(prompt, request_options=retry_policy)
print(response.text)

Here's how to solve this:

* **When you were 4, your partner was 3 times your age:** 4 years old * 3 = 12 years old.
* **The age difference between you and your partner:** 12 years old - 4 years old = 8 years.
* **Since the age difference remains constant, your partner is still 8 years older than you.**
* **Your partner's current age:** 20 years old + 8 years = 28 years old.

**Therefore, your partner is 28 years old.** 



### ReAct: Reason and act

In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the chapter.

To try this out with the Wikipedia search engine, check out the [Searching Wikipedia with ReAct](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) cookbook example.


> Note: The prompt and in-context examples used here are from [https://github.com/ysymyth/ReAct](https://github.com/ysymyth/ReAct) which is published under a [MIT license](https://opensource.org/licenses/MIT), Copyright (c) 2023 Shunyu Yao.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>


In [34]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

To capture a single step at a time, while ignoring any hallucinated Observation steps, you will use `stop_sequences` to end the generation process. The steps are `Thought`, `Action`, `Observation`, in that order.

In [35]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
react_chat = model.start_chat()

# You will perform the Action, so generate up to, but not including, the Observation.
config = genai.GenerationConfig(stop_sequences=["\nObservation"])

resp = react_chat.send_message(
    [model_instructions, example1, example2, question],
    generation_config=config,
    request_options=retry_policy)
print(resp.text)

Thought 1
The question asks for the youngest author of the Transformers NLP paper. I should search for the paper, find the author list, and then determine the youngest author.

Action 1
<search>Transformers NLP paper</search>



Now you can perform this research yourself and supply it back to the model.

In [36]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation, generation_config=config, request_options=retry_policy)
print(resp.text)

Thought 2
The observation contains the author list, but it does not contain their ages. There is no information about the age of the authors in the observation. I need to find information about the ages of these authors. I can try searching each author individually.

Action 2
<search>Ashish Vaswani age</search> 



This process repeats until the `<finish>` action is reached. You can continue running this yourself if you like, or try the [Wikipedia example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb) to see a fully automated ReAct system at work.

## Code prompting

### Generating code

The Gemini family of models can be used to generate code, configuration and scripts. Generating code can be helpful when learning to code, learning a new language or for rapidly generating a first draft.

It's important to be aware that since LLMs can't reason, and can repeat training data, it's essential to read and test your code first, and comply with any relevant licenses.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1YX71JGtzDjXQkgdes8bP6i3oH5lCRKxv"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [37]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ))

# Gemini 1.5 models are very chatty, so it helps to specify they stick to the code.
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = model.generate_content(code_prompt, request_options=retry_policy)
Markdown(response.text)

```python
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```

### Code execution

The Gemini API can automatically run generated code too, and will return the output.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [ ]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    tools='code_execution',)

code_exec_prompt = """
Calculate the sum of the first 14 prime numbers. Only consider the odd primes, and make sure you count them all.
"""

response = model.generate_content(code_exec_prompt, request_options=retry_policy)
Markdown(response.text)

While this looks like a single-part response, you can inspect the response to see the each of the steps: initial text, code generation, execution results, and final text summary.

In [ ]:
for part in response.candidates[0].content.parts:
  print(part)
  print("-----")

### Explaining code

The Gemini family of models can explain code to you too.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1N7LGzWzCYieyOf_7bAG4plrmkpDNmUyb"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [ ]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')

response = model.generate_content(explain_prompt, request_options=retry_policy)
Markdown(response.text)

## Learn more

To learn more about prompting in depth:

* Check out the whitepaper issued with today's content,
* Try out the apps listed at the top of this notebook ([TextFX](https://textfx.withgoogle.com/), [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) and [NotebookLM](https://notebooklm.google/)),
* Read the [Introduction to Prompting](https://ai.google.dev/gemini-api/docs/prompting-intro) from the Gemini API docs,
* Explore the Gemini API's [prompt gallery](https://ai.google.dev/gemini-api/prompts) and try them out in AI Studio,
* Check out the Gemini API cookbook for [inspirational examples](https://github.com/google-gemini/cookbook/blob/main/examples/) and [educational quickstarts](https://github.com/google-gemini/cookbook/blob/main/quickstarts/).

And please share anything exciting you have tried in the Discord!